In [6]:
fresh_start_ollama()

Stopping Ollama service...
Starting Ollama service...
Ollama service successfully restarted


True

In [10]:
import subprocess
import time
from typing import Optional

def cleanup_gpu_memory():
    """
    Clean up GPU memory using nvidia-smi.
    """
    try:
        # Clear GPU memory using nvidia-smi
        subprocess.run(['nvidia-smi', '--gpu-reset'], check=True)
        print("GPU memory reset completed")
        
        # Get current GPU stats
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        print("\nGPU Statistics after cleanup:")
        print(result.stdout)
            
    except Exception as e:
        print(f"Error during GPU memory cleanup: {str(e)}")

def fresh_start_ollama(wait_time: int = 5) -> bool:
    """
    Restarts the Ollama service and cleans up GPU memory.
    
    Args:
        wait_time (int): Time to wait in seconds after restarting Ollama
    
    Returns:
        bool: True if successful, False if an error occurred
    """
    try:
        # First, clean up GPU memory
        print("Cleaning up GPU memory...")
        cleanup_gpu_memory()
        
        # Stop Ollama service
        print("Stopping Ollama service...")
        subprocess.run(['sudo', 'systemctl', 'stop', 'ollama'], check=True)
        
        # Additional GPU cleanup after service stop
        cleanup_gpu_memory()
        
        time.sleep(2)  # Wait for complete shutdown
        
        # Start Ollama service
        print("Starting Ollama service...")
        subprocess.run(['sudo', 'systemctl', 'start', 'ollama'], check=True)
        
        # Wait for service to be ready
        time.sleep(wait_time)
        
        # Verify service is running
        result = subprocess.run(['systemctl', 'is-active', 'ollama'], 
                              capture_output=True, 
                              text=True)
        
        if result.stdout.strip() == 'active':
            print("Ollama service successfully restarted")
            return True
        else:
            print("Warning: Ollama service might not be fully active")
            return False
            
    except Exception as e:
        print(f"Error during Ollama restart: {str(e)}")
        return False

def nvidia_smi_stats():
    """
    Get NVIDIA GPU statistics using nvidia-smi.
    """
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        print("\nGPU Statistics:")
        print(result.stdout)
    except Exception as e:
        print(f"Error getting GPU stats: {str(e)}")

# Example benchmark function with monitoring
def benchmark_with_monitoring(model_name: str, warm_up_prompt: str, actual_prompt: str):
    """
    Run a benchmark with GPU monitoring.
    """
    import ollama
    
    print("\nStarting fresh with GPU cleanup...")
    fresh_start_ollama(wait_time=5)
    
    print("\nGPU state before warm-up:")
    nvidia_smi_stats()
    
    print("\nRunning warm-up query...")
    start_time = time.time()
    response = ollama.chat(model=model_name, messages=[
        {'role': 'user', 'content': warm_up_prompt}
    ])
    warm_up_time = time.time() - start_time
    print(f"Warm-up time: {warm_up_time:.2f} seconds")
    
    print("\nGPU state after warm-up:")
    nvidia_smi_stats()
    
    # Small delay between queries
    time.sleep(2)
    
    print("\nRunning actual query...")
    start_time = time.time()
    response = ollama.chat(model=model_name, messages=[
        {'role': 'user', 'content': actual_prompt}
    ])
    query_time = time.time() - start_time
    print(f"Query time: {query_time:.2f} seconds")
    
    print("\nFinal GPU state:")
    nvidia_smi_stats()
    
    return {
        'warm_up_time': warm_up_time,
        'query_time': query_time,
        'response': response['message']['content']
    }

In [11]:
fresh_start_ollama()

Cleaning up GPU memory...
GPU Reset couldn't run because GPU 00000000:01:00.0 is the primary GPU.
Error during GPU memory cleanup: Command '['nvidia-smi', '--gpu-reset']' returned non-zero exit status 3.
Stopping Ollama service...
GPU Reset couldn't run because GPU 00000000:01:00.0 is the primary GPU.
Error during GPU memory cleanup: Command '['nvidia-smi', '--gpu-reset']' returned non-zero exit status 3.
Starting Ollama service...
Ollama service successfully restarted


True

In [8]:
nvidia_smi_stats()

NameError: name 'nvidia_smi_stats' is not defined